In [192]:
import csv
import json
import numpy as np
import urllib
from datetime import datetime, timedelta
import time


def write_results(result_list,csv_out_file):

    with open(csv_out_file, 'w', newline='') as f:
        writer = csv.writer(f, delimiter = ' ')
        for elem in result_list:
            writer.writerow(elem)

def pull_rki_data():
    import urllib.request, json 
    from datetime import timedelta, date

    def daterange(start_date, end_date):
        for n in range(int ((end_date - start_date).days)):
            yield start_date + timedelta(n)

    start_date = date(2020, 3, 1)
    end_date = date(2020, 3, 1)+ timedelta(days=1)
    
    #json_comp =json.dumps({})
    json_comp = []

    for single_date in daterange(start_date, end_date):
        with urllib.request.urlopen("https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0//query?where=Meldedatum%3D%27"+str(single_date.strftime("%Y-%m-%d"))+"%27&objectIds=&time=&resultType=none&outFields=*&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&sqlFormat=none&f=pjson&token=") as url:
            json_data = json.loads(url.read().decode("utf-8"))
            json_comp.append(json_data)# json_comp.append(json_data)
            
    #print("",list_data)
    return(json.dumps(json_comp))


def load_data(csvfile):
    
    res = urllib.request.urlopen('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')
    res_body = res.read()

    data = []
    population_list = []
    landkreis_list= []
      
    with open(csvfile, 'r') as landkreis_file:
            reader = csv.reader(landkreis_file, delimiter=';')
            line = landkreis_file.readline()
            for line in reader:
                population_list.append(line[9:12])
                    

    if filepath.endswith(".json"):
        with open(filepath) as json_file:
            arcgis = json.loads(res_body.decode("utf-8"))
            #arcgis =  pull_rki_data()
            print("",arcgis)
            
            #Go over all landkreise in arcgis data and extract all different names
            landkreise = []
            if 'features' in arcgis and arcgis['features']:
                for feature in arcgis['features']:
                        if not any(x == feature['attributes']['IdLandkreis'] for x in landkreise):
                            if not feature['attributes']['IdLandkreis'] == '0-1':
                                landkreise.append(feature['attributes']['IdLandkreis'])


                
    print("Number of landkreise",len(landkreise))
    print("Number of population",len(population_list))
    
    
     
    # for each landkreis estimate the earliest occurence data ---> then get all cases for this day     
    for landkreis in landkreise:

            occurrence_date = 0
            first_occurrence = 0
            first_alert_date = []
            number_of_cases_on_first_date = 0
            number_of_cases = 0
            pop = 0
            nuts3 = ''

            if 'features' in arcgis and arcgis['features']:
                for feature in arcgis['features']:
                    total_case_numbers = 0

                    if landkreis == feature['attributes']['IdLandkreis']: 
                        alert_date = feature['attributes']['Meldedatum'] 
                        diff_now_alert_date = 1584835200000 - alert_date #date of today at midnight serves as reference TODO change to date of today dynamically
                        diff_now_alert_date = diff_now_alert_date/(1000*60*60*24)
                        
                        occurrence_date = int(diff_now_alert_date)

                        if occurrence_date > first_occurrence:
                            first_occurrence = occurrence_date
                            first_alert_date =feature['attributes']['Meldedatum'] 

            if 'features' in arcgis and arcgis['features']:
                for feature in arcgis['features']:   
                    if landkreis == feature['attributes']['IdLandkreis']:
                        number_of_cases  = number_of_cases + feature['attributes']['AnzahlFall']
                        if first_alert_date == feature['attributes']['Meldedatum']:
                            number_of_cases_on_first_date = number_of_cases_on_first_date + feature['attributes']['AnzahlFall']


            for line in population_list:
                if not line[0] == '0-1':
                    if int(line[0]) == int(landkreis):
                        pop = int(line[1])
                        nuts3 = line[2]

            landkreis_list.append([int(landkreis),number_of_cases_on_first_date,number_of_cases,first_occurrence, pop, nuts3])
            
    return landkreis_list
            
csvfile = 'C:/Users/thomass/Desktop/RKI-COVID-mit-Einwohnerzahl_und_NUTS3_pro_Fall.csv'
landkreis_list = load_data(csvfile)

write_results(landkreis_list,'C:/Users/thomass/Desktop/RKI-COVID_out.csv')

print("",landkreis_list)
# list with: 
# Landkreis ID
# number_of_cases_on_first_date
# number_of_cases_now
# first_occurrence in days
# population
# nuts3






 {'objectIdFieldName': 'ObjectId', 'uniqueIdField': {'name': 'ObjectId', 'isSystemMaintained': True}, 'globalIdFieldName': '', 'fields': [{'name': 'IdBundesland', 'type': 'esriFieldTypeInteger', 'alias': 'IdBundesland', 'sqlType': 'sqlTypeInteger', 'domain': None, 'defaultValue': None}, {'name': 'Bundesland', 'type': 'esriFieldTypeString', 'alias': 'Bundesland', 'sqlType': 'sqlTypeNVarchar', 'length': 2147483647, 'domain': None, 'defaultValue': None}, {'name': 'Landkreis', 'type': 'esriFieldTypeString', 'alias': 'Landkreis', 'sqlType': 'sqlTypeNVarchar', 'length': 2147483647, 'domain': None, 'defaultValue': None}, {'name': 'Altersgruppe', 'type': 'esriFieldTypeString', 'alias': 'Altersgruppe', 'sqlType': 'sqlTypeNVarchar', 'length': 2147483647, 'domain': None, 'defaultValue': None}, {'name': 'Geschlecht', 'type': 'esriFieldTypeString', 'alias': 'Geschlecht', 'sqlType': 'sqlTypeNVarchar', 'length': 2147483647, 'domain': None, 'defaultValue': None}, {'name': 'AnzahlFall', 'type': 'esriFi

 [[1001, 4, 13, 8, 89504, 'DEF01'], [1002, 3, 39, 12, 247548, 'DEF02'], [1003, 1, 21, 22, 217198, 'DEF03'], [1004, 1, 7, 6, 79487, 'DEF04'], [1051, 1, 12, 17, 133210, 'DEF05'], [1053, 1, 21, 20, 197264, 'DEF06'], [1054, 1, 17, 10, 165507, 'DEF07'], [1055, 1, 12, 12, 200581, 'DEF08'], [1056, 1, 77, 19, 314391, 'DEF09'], [1057, 2, 12, 13, 128647, 'DEF0A'], [1058, 2, 33, 11, 272775, 'DEF0B'], [1059, 1, 25, 11, 200025, 'DEF0C'], [1060, 1, 29, 23, 276032, 'DEF0D'], [1061, 2, 12, 9, 131347, 'DEF0E'], [1062, 2, 17, 10, 243196, 'DEF0F'], [2000, 1, 872, 22, 1841179, 'DE600'], [3101, 1, 25, 17, 248292, 'DE911'], [3102, 1, 16, 11, 104948, 'DE912'], [3103, 3, 15, 9, 124151, 'DE913'], [3151, 2, 27, 11, 175920, 'DE914'], [3153, 5, 18, 9, 137014, 'DE916'], [3154, 2, 16, 9, 91307, 'DE917'], [3155, 3, 12, 11, 132765, 'DE918'], [3157, 2, 16, 9, 133965, 'DE91A'], [3158, 1, 3, 5, 119960, 'DE91B'], [3159, 4, 51, 10, 328074, 'DE91C'], [3241, 1, 233, 21, 1157624, 'DE929'], [3251, 1, 61, 12, 216886, 'DE922'],

In [177]:
landkreis_list

<_io.TextIOWrapper name='C:/Users/thomass/Desktop/Einwohnerzahl.csv' mode='r' encoding='cp1252'>